<a href="https://colab.research.google.com/github/adenehemia09/Alia-AI-Chatbot/blob/main/alia_ai_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import string
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import json

In [ ]:
import json
import pandas as pd
from random import choice


class JSONParser:
    def __init__(self):
        self.text = []
        self.intents = []
        self.responses = {}

    def parse(self, json_path):
        with open(json_path) as data_file:
            self.data = json.load(data_file)

        for intent in self.data['intents']:
            for pattern in intent['patterns']:
                self.text.append(pattern)
                self.intents.append(intent['tag'])
            for resp in intent['responses']:
                if intent['tag'] in self.responses.keys():
                    self.responses[intent['tag']].append(resp)
                else:
                    self.responses[intent['tag']] = [resp]

        self.df = pd.DataFrame({'text_input': self.text,
                                'intents': self.intents})

        print(
            f"[INFO] Data JSON converted to DataFrame with shape : {self.df.shape}")

    def get_dataframe(self):
        return self.df

    def get_response(self, intent):
        return choice(self.responses[intent])

In [ ]:
# load data
path = "/content/drive/MyDrive/ Bisa AI/NLP/Chat Bot/new_intents.json"

# buat objek JSONParser dan parse data intents.json
jp = JSONParser()
jp.parse(path)

# simpan dataframe dalam variabel df
df = jp.get_dataframe()

[INFO] Data JSON converted to DataFrame with shape : (134, 2)


In [ ]:
# lihat 5 data pertama
df.head(10)

,text_input,intents
0,Hai,salam
1,Hi,salam
2,Halo,salam
3,Helo,salam
4,Hello,salam
5,Hallo,salam
6,P,salam
7,Ping,salam
8,Nama kamu siapa?,nama
9,lu siapa sih?,nama


In [ ]:
# hitung jumlah data per tag / inten
df.intents.value_counts()

bye                                14
nama                               10
kulit-berjerawat                    9
rambut                              9
kulit-berminyak-dan-kulit-kusam     8
salam                               8
nama-toko                           7
komplaintime                        7
salahkirim                          7
kemampuan                           6
super-sale                          6
tanyaWaktuPenggunaanMasker          6
tanyaisiserum                       6
barangrusak                         6
infotoko                            5
oil-control                         5
tanyaPenggunaanMasker               5
tanyapemakaianserum                 5
serumAndMask                        5
Name: intents, dtype: int64

In [ ]:
def preprocess(chat):
    """
    Fungsi yang digunakan untuk melakukan praproses
    """
    # konversi ke lowercase
    chat = chat.lower()
    # menghapus tanda baca
    tandabaca = tuple(string.punctuation)
    chat = ''.join(ch for ch in chat if ch not in tandabaca)
    return chat

In [ ]:
# implementasikan fungsi preprocess ke string
df['text_input_prep'] = df.text_input.apply(preprocess)

In [ ]:
df[['text_input', 'text_input_prep']].head(10)

,text_input,text_input_prep
0,Hai,hai
1,Hi,hi
2,Halo,halo
3,Helo,helo
4,Hello,hello
5,Hallo,hallo
6,P,p
7,Ping,ping
8,Nama kamu siapa?,nama kamu siapa
9,lu siapa sih?,lu siapa sih


In [ ]:
# inisiasi objek CountVectorizer
vect = CountVectorizer()

In [ ]:
# mengumpulkan vocab dari data teks yang sudah dilakukan praproses
vect.fit(df['text_input_prep'])

CountVectorizer()

In [ ]:
# lihat list vocab
vect.get_feature_names()[:20] #batasi hanya 10 vocab teratas

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['ad',
 'ada',
 'adakah',
 'aj',
 'aja',
 'ajuran',
 'alia',
 'anjuran',
 'ap',
 'apa',
 'apakah',
 'atas',
 'atau',
 'bagaimana',
 'banget',
 'banrang',
 'barang',
 'barangnya',
 'beraktifitas',
 'berapa']

In [ ]:
# ubah data teks menjadi matriks
text_vect = vect.transform(df.text_input_prep)

text_vect

<134x160 sparse matrix of type '<class 'numpy.int64'>'
	with 642 stored elements in Compressed Sparse Row format>

In [ ]:
# deklarasi objek MultinomialNB
nb = MultinomialNB()

# training data, dengan X : text_vect dan y : intents
nb.fit(text_vect, df.intents)

MultinomialNB()

In [ ]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict(chat)

# tampilkan hasil prediksi
print(f"Hasil prediksi : {res[0]}")

Masukkan String : hi
Hasil prediksi : salam


In [ ]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict_proba(chat)

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

Masukkan String : nama lu siape?
Max Prob : 0.6002995097051386
Max Index: 7
Label: nama


In [ ]:
# Deklarasi pipeline yang mengandung vektorisasi (CountVectorizer) & pemodelan (MultinomialNB) 
pipe = make_pipeline(CountVectorizer(),
                     MultinomialNB())

# Training
pipe.fit(df.text_input, df.intents)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [ ]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# prediksi teks kedalam pipeline
res = pipe.predict_proba([chat])

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

Masukkan String : hi
Max Prob : 0.12991580615160164
Max Index: 12
Label: salam


In [ ]:
print("Garnier Men Official Store")
print("Anda Terhubung dengan chatbot Kami")
while True:
    # input user
    chat = input("Anda : ")
    # praproses
    chat = preprocess(chat)
    # prediksi intent
    res = pipe.predict_proba([chat])
    # ambil nilai probabilitas & lokasinya
    max_prob = max(res[0])
    max_idx = np.argmax(res[0])
    # kondisi jika probabilitas kurang dari threshold
    if max_prob < 0.10:
        print("Bot : Maaf Kak, aku ga ngerti")
    else:
        print(f"Bot : {jp.get_response(nb.classes_[max_idx])}")
    if nb.classes_[max_idx] == 'bye':
        break

Garnier Men Official Store
Anda Terhubung dengan chatbot Kami
Anda : hallo
Bot : Hallo kak, ada yang bisa saya bantu?
Anda : nama lu siapa sih?
Bot : Saya "Alia" Yah, semacam nama panggilan.
Anda : jam brp tokonya buka, min"
Bot : Sahabat garnier, Terima kasih sudah menunggu respon kami.
     Toko beroprasi pukul "08:00-11:00 WIB". Pesanan yang lewat pukul "11:00" akan dioprasi kembali.
     Semoga informasi ini bisa membantu ya, sobat. Terima kasih.
Anda : LU bisa apa aja?
Bot : Kakak bisa membeli produk "Garnier Men" dengan menuliskan nama produk!
Anda : Produk apa yang cocok buat wajah yang kusam dan berminyak?
Bot : Baik kak, Untuk "Kulit berminya dan kusam" bisa kunjungi etalase "Garnier Men" di https://www.tokopedia.com/garniermen/etalase/kulit-berminyak-dan-kulit-kusam 
Anda : kalau untuk kulit berjerawat bagaimana?
Bot : Untuk masalah "kulit berjerawat", bisa kunjungi tokoh kami di https://www.tokopedia.com/garniermen/etalase/kulit-berjerawat
Anda : saya punya masalah kulit yan